In [1]:
!nvidia-smi

Sat Feb 27 10:29:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install pdf2image

In [4]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (309 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 149406 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [5]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Auto Grade

Mounted at /content/drive
Mounted at /gdrive
/gdrive/My Drive/Auto Grade


In [6]:
from pdf2image import convert_from_path
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image 
import numpy as np

from keras.models import load_model
model = load_model('cnn.h5')

In [7]:
def get_answers(file):
  pages = convert_from_path(file)
  # pages = convert_from_path('test-multipage.pdf')
  answers = []
  for page in pages:
      page.save('out.jpg', 'JPEG')
      image = cv2.imread('out.jpg')
      cim = cv2.imread('out.jpg')
      imgGrey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      _, thrash = cv2.threshold(imgGrey, 240, 255, cv2.THRESH_BINARY)
      contours, _ = cv2.findContours(thrash, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      count=0
      rectangles = []
      for contour in contours:
          approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
          cv2.drawContours(image, [approx], 0, (0, 0, 0), 5)
          x = approx.ravel()[0]
          y = approx.ravel()[1] - 5
          if len(approx) == 4:
              x1 ,y1, w, h = cv2.boundingRect(approx)
              aspectRatio = float(w)/h
              if aspectRatio >= 0.95 and aspectRatio <= 1.05:
                pass
              else:
                if(x1!=0 and h>30):
                  rectangles.append([x1 ,y1, w, h])
                  count=count+1

      px1 ,py1, pw, ph = rectangles[0]
      final = [rectangles[0]]

      for rectangle in rectangles:
        if(py1-ph>rectangle[1]):
          px1 ,py1, pw, ph = rectangle
          final.append(rectangle)
      for i in final:
        cv2.rectangle(image,(i[0],i[1]),(i[0]+i[2],i[1]+i[3]),(0,255,0),6)
      # cv2_imshow(image)
      # print(final)
      page_answers=[]
      for i in final:
        img = cim[i[1]:i[1]+i[3], i[0]:i[0]+i[2]]
        # cv2_imshow(img)
        img = Image.fromarray(img) 
        x=img.convert("L") 
        x = x.resize((28,28))
        x.show()
        #compute a bit-wise inversion so black becomes white and vice versa
        x = np.invert(x)
        #convert to a 4D tensor to feed into our model
        x = x.reshape(1,28,28,1)
        x = x.astype('float32')
        x /= 255
        out = model.predict(x)
        # print(np.argmax(out[0][1:5])+1)
        page_answers.append(np.argmax(out[0][1:5])+1)
      answers.extend(page_answers[::-1])
  return answers


In [8]:
get_answers("test-multipage.pdf")

[2, 1, 3, 4, 3, 4, 1, 1, 1, 2, 3, 2, 2, 3, 3]